In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark\\spark-3.0.0-preview-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('International_Trip Purpose').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
spark

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date
from pyspark.sql.functions import month
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.functions import round,array,col,ntile,rank,asc,sum,desc,lag,lead,when,first,min,max,count
# For Adding row number/index
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window


#Import Library
#import findspark
#findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark import SparkContext, SparkConf, keyword_only
from pyspark.sql import SparkSession

#from pyspark.sql import SQLContext, HiveContext, SparkSession
from pyspark.sql import Row

In [5]:
trip_level2=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\trip_mode_nov\*.csv",inferSchema='true')

In [6]:
Holiday_cal=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\Holiday_cal.csv",inferSchema='true')

In [7]:
trip_level2=trip_level2.withColumn('imsi_trip_id', concat(col('imsi'),lit('_'),col('trip_id')))

In [8]:
#Holiday_cal.show(4),Holiday_cal.printSchema()

In [9]:
#trip_level2.select('imsi','trip_id','trip_start_time','trip_end_time').show()

In [10]:
trip_level3=trip_level2.where(~col('trip_start_time').contains("Trip_start_missing") & ~col('trip_end_time').contains("Trip_end_missing")).select('imsi','trip_id','trip_start_time','trip_end_time')

In [11]:
trip_level3.show()

+----+-------+---------------+-------------+
|imsi|trip_id|trip_start_time|trip_end_time|
+----+-------+---------------+-------------+
+----+-------+---------------+-------------+



In [12]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import month

In [13]:
Holiday_cal=Holiday_cal.withColumn('date_id',to_date(Holiday_cal.date_id, 'dd-MM-yyyy'))
#Holiday_cal=Holiday_cal.withColumn('month',month(Holiday_cal.date_id))

In [14]:
from pyspark.sql.functions import to_date
from pyspark.sql import functions as F
trip_level3=trip_level3.withColumn('trip_start_time',to_timestamp(col('trip_start_time'), 'dd-MM-yyyy HH:mm'))\
            .withColumn('trip_end_time',to_timestamp(col('trip_end_time'), 'dd-MM-yyyy HH:mm'))

In [15]:
trip_level3=trip_level3.withColumn('start_date',to_date(col('trip_start_time'), 'yyyy-MM-dd HH:mm:ss'))\
            .withColumn('end_date',to_date(col('trip_end_time'), 'yyyy-MM-dd HH:mm:ss'))

In [16]:
trip_level3=trip_level3.withColumn('DiffInSeconds',unix_timestamp(col('trip_end_time')) - unix_timestamp(col('trip_start_time'))) 

In [17]:
trip_level3=trip_level3.withColumn('trip_duration_min',round(col('DiffInSeconds')/60))

In [18]:
trip_level3.show()

+----+-------+---------------+-------------+----------+--------+-------------+-----------------+
|imsi|trip_id|trip_start_time|trip_end_time|start_date|end_date|DiffInSeconds|trip_duration_min|
+----+-------+---------------+-------------+----------+--------+-------------+-----------------+
+----+-------+---------------+-------------+----------+--------+-------------+-----------------+



In [19]:
from pyspark.sql import functions as F

df = trip_level3 \
  .withColumn('start_date1', F.col('start_date').cast('date')) \
  .withColumn('end_date1', F.col('end_date').cast('date'))

trip_level4=trip_level3.withColumn('txnDt', F.explode(F.expr('sequence(start_date, end_date, interval 1 day)'))).dropDuplicates()

In [20]:
trip_level5=trip_level4.join(Holiday_cal.select('date_id'),(trip_level4.txnDt==Holiday_cal.date_id),'left')

In [21]:
### Extract day of week from date in pyspark
from pyspark.sql.functions import dayofweek

df1 = trip_level5.withColumn('day_of_week',dayofweek(trip_level5.txnDt))
#df1.show(2)

In [22]:
df1 = df1.withColumn("Holiday", when(col("date_id").isNull(),lit(0)).otherwise(lit(1)))
df1 = df1.withColumn("weekend", when(col("day_of_week")>5,lit(1)).otherwise(lit(0)))

In [23]:
df_agg = df1.groupBy('imsi','trip_id','trip_start_time','trip_end_time','start_date','end_date','trip_duration_min').agg(sum('Holiday').alias('Holiday'),
                                                                                                    sum("weekend").alias("weekend"))

In [24]:
trip_purpuse=trip_level2.join(df_agg.select('imsi','trip_id','Holiday','weekend','trip_duration_min'),['imsi','trip_id'],'left')

In [25]:
trip_purpuse1=trip_purpuse.withColumn("trip_purpuse", when(col("Holiday")!=0,'Holiday_trip')\
                                      .when(((col("Holiday")==0) & (col('weekend')>0)),'Weekend_trip')\
                                      .when((col("trip_start_time") == 'Trip_start_missing') | (col("trip_end_time").isNull()),'')\
                                      .otherwise('Business_trip'))

In [26]:
trip_purpuse1=trip_purpuse1.select('imsi','trip_id','trip_start_time','trip_end_time','trip_duration_min','exit_grid_id','exit_grid_name','entry_grid_id','entry_grid_name',
 'exit_Mode','entry_Mode','trip_purpuse')

In [27]:
#trip_purpuse1=trip_purpuse1.withColumn("trip_duration_min", round(trip_purpuse1["trip_duration_min"], 2).cast('integer'))

In [28]:
trip_purpuse1.repartition(1).write.option("header",True).mode("overwrite").csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\trip_purpuse_nov")

### Trip cut off

In [29]:
trip_purpuse=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\trip_purpuse_nov\*.csv",inferSchema='true')

In [30]:
trip_purpuse.printSchema()

root
 |-- imsi: string (nullable = true)
 |-- trip_id: integer (nullable = true)
 |-- trip_start_time: string (nullable = true)
 |-- trip_end_time: string (nullable = true)
 |-- trip_duration_min: string (nullable = true)
 |-- exit_grid_id: string (nullable = true)
 |-- exit_grid_name: string (nullable = true)
 |-- entry_grid_id: string (nullable = true)
 |-- entry_grid_name: string (nullable = true)
 |-- exit_Mode: string (nullable = true)
 |-- entry_Mode: string (nullable = true)
 |-- trip_purpuse: string (nullable = true)



In [31]:
from pyspark.sql.functions import when, col
trip_cut_off = trip_purpuse.withColumn("start_available", when(col("trip_start_time") == 'Trip_start_missing',0).otherwise(1))

In [32]:
trip_cut_off = trip_cut_off.withColumn("end_available", when(col("trip_end_time")== 'Trip_end_missing',0).otherwise(1))

In [33]:
trip_cut_off = trip_cut_off.withColumn("incomplete_trip", when((col("start_available")==1) & (col("end_available")==1),0).otherwise(1))

In [34]:
trip_cut_off.repartition(1).write.option("header",True).mode("overwrite").csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\trip_cut_off_nov")

In [35]:
spark.stop()